In [ ]:
import numpy as np
import random
import Scenarios, Evaluator
import json
import time

In [ ]:
sc=Scenarios.Scenario()
eval=Evaluator.Evaluator()
angles, obstacles, width, height, nturbines, wfe=sc.getScenario("3.xml")
sc.initOptpar()
rows=0
cols=0
minRad=79
solMat=np.empty((0,0))
solR=[]
solC=[]
interval=0.1*sc.R

In [ ]:
def getDatafromFile():
    data=0
    with open("drive/MyDrive/Dissertation/scenario4Data.json") as f:
        data=json.load(f)
    grid=data["Grid"]
    print("Grid Collected")
    solMat=data["SolMat"]
    print("SolMat Collected")
    rows=data["rows"]
    print("Rows Collected")
    cols=data["cols"]
    print("Cols Collected")
    solR=data["SolR"]
    print("SolR Collected")
    solC=data["SolC"]
    print("SolC Collected")
    print("Data Collected")
    
    return grid,solMat,rows,cols,solR,solC

In [ ]:

# Sample grid generator
def createGrid():
    grid=[]
    rows=0
    cols=0
    for x in np.arange(0,width,interval):
        rows+=1
    for x in np.arange(0,height,interval):
        cols+=1
    solMat=np.empty((rows,cols))
    print(solMat.shape)
    
    gridIdx=0
    curRow=-1
    for x in np.arange(0,width,interval):
        curCol=-1
        curRow+=1
        for y in np.arange(0,height,interval):
            curCol+=1
            valid=True
            for i in range(len(obstacles)):
                obs=obstacles.loc[i]
                if(x>obs["xmin"] and y>obs["ymin"] and x<obs["xmax"] and y<obs["ymax"]):
                    valid=False
            if (valid):
                point=(x,y)
                grid.append(point)
                solMat[curRow][curCol]=int(gridIdx)
                gridIdx+=1
            else:
                solMat[curRow][curCol]=int(-1)
    
    return grid,solMat,rows,cols
#grid,solMat=createGrid()

In [ ]:
def solPar():
    solR=[0]*len(grid)
    solC=[0]*len(grid)
    gridIdx=0
    curRow=-1
    for x in np.arange(0,width,interval):
        curCol=-1
        curRow+=1
        for y in np.arange(0,height,interval):
            curCol+=1
            valid=True
            for i in range(len(obstacles)):
                obs=obstacles.loc[i]
                if(x>obs["xmin"] and y>obs["ymin"] and x<obs["xmax"] and y<obs["ymax"]):
                    valid=False
            if (valid):
                solR[gridIdx]=curRow
                solC[gridIdx]=curCol
                gridIdx+=1
    
    return solR,solC
#solR,solC=solPar()

In [ ]:
def initialSol(x,y,sh):
    solution=[False]*len(grid)
    for i in range(0,rows,minRad+1+x):
        for j in range(0,cols,minRad+1+y):
            curShift=(j/(minRad+1+y))*sh
            needfix=False
            curShift=int(i+curShift)
            if(curShift>=rows):
                needfix=True
                curShift=curShift%rows
            if(solMat[curShift][j]!=-1):
                solution[int(solMat[curShift][j])]=True
                if(needfix):
                    solution=fixPos(int(solMat[curShift][j]),solution)
    print("X: ",x," Y: ",y," Shift: ",sh)
    return solution

In [ ]:
from os import remove
def createSol(llhs,bestllh):
    x=0
    y=0
    sh=0
    
    llh=llhs+bestllh

    for up in llh:
        if up==0:
            x=incX(x)
        elif up==1:
            x=decX(x)
        elif up==2:
            y=incY(y)
        elif up==3:
            y=decY(y)
        elif up==4:
            sh=incShift(sh)
        elif up==5:
            sh=decShift(sh)
    
    solution=[False]*len(grid)
    for i in range(0,rows,minRad+1+x):
        for j in range(0,cols,minRad+1+y):
            curShift=(j/(minRad+1+y))*sh
            needfix=False
            if sh<0:
                needfix=True
            curShift=int(i+curShift)
            if(curShift>=rows):
                needfix=True
                curShift=curShift%rows
            if(solMat[curShift][j]!=-1):
                solution[int(solMat[curShift][j])]=True
                if(needfix):
                    solution=fixPos(int(solMat[curShift][j]),solution)

    for up in llh:
        if (up==6):
              solution=removeTurbine(solution)
        elif (up==7):
              solution=removeStrip(solution)
#     if up==0:
#         solution=incX(solution)
#     elif up==1:
#         solution=decX(solution)
#     elif up==2:
#         solution=incY(solution)
#     elif up==3:
#         solution=decY(solution)
#     elif up==4:
#         solution=incshiftRagged(solution)
#     elif up==5:
#         solution=decshiftRagged(solution)
#     if (up==6):
#       solution=removeTurbine(solution)
# #     elif (up==7):
# #       solution=addTurbine(solution)
#     elif (up==7):
#       solution=removeStrip(solution)
#     elif (up==8):
#       solution=removeStripAdd(solution)

    return solution

In [ ]:
def fixPos(idx,solution):
    if(minRad>0):
        if(solution[idx]):
            curRow=solR[idx]
            curCol=solC[idx]
            for i in range(curRow-minRad,curRow+minRad):
                for j in range(curCol-minRad,curCol+minRad):
                    if ( (i >= 0) and (i < rows) and (j >= 0) and (j < cols) ):
                        if(not ((i==curRow) and (j==curCol))):
                            if(solMat[i][j] != -1):
                                if(solution[int(solMat[i][j])]):
                                    solution[idx]=False
    return solution
    

In [ ]:
def evaluate(sol):
    #check all zeros
    eval=Evaluator.Evaluator()
    eval.initialise(sc)
    allZeros=True
    for i in range(0,len(grid)):
        if(sol[i]):
            allZeros=True
            break
    nturbines=0
    for i in range(len(grid)):
        if(sol[i]):
            nturbines+=1
    layout=np.empty((nturbines,2))
    l_i=0
    for i in range(len(grid)):
        if(sol[i]):
            layout[l_i][0]=grid[i][0]
            layout[l_i][1]=grid[i][1]
            l_i+=1
    return(eval.evaluate(layout))
    

In [ ]:
#Use this code segment if creating grid for first time. 
grid,solMat,rows,cols=createGrid()
solR,solC=solPar()

#creating Json file to store created grid points for furure use.
import json
dict={"Grid":grid,"SolMat":solMat.tolist(),"rows":rows,"cols":cols,"SolR":solR,"SolC":solC}
json_object = json.dumps(dict, indent = 6)
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
#Use this to load grid from previously created Json file
grid,solMat,rows,cols,solR,solC=getDatafromFile()
solMat=np.array(solMat)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def visualize(mysolution):
    x=0
    newLayout=[]
    for i in range(rows):
        for j in range(cols):
            if mysolution[x]:
                newLayout.append(grid[x])
            if x<len(mysolution)-1:
                x+=1
    plt.figure(figsize=(20,20))
    plt.ylim(height+100,-50)
    plt.xlim(-50,width+100)
    

    farm=mpatches.Rectangle((0,0),width,height, 
                            fill = False,
                            color = "black",
                            linewidth = 2)

    plt.gca().add_patch(farm)
    for i in range(len(obstacles)):
        xmin=obstacles.loc[i]["xmin"]
        xmax=obstacles.loc[i]["xmax"]
        ymin=obstacles.loc[i]["ymin"]
        ymax=obstacles.loc[i]["ymax"]
        rect1=mpatches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin), 
                            fill = False,
                            color = "purple",
                            linewidth = 2)
        plt.gca().add_patch(rect1)
    # plt.plot(10,10,marker="o", markersize=20, markeredgecolor="red", markerfacecolor="green")
    # plt.plot(newLayout[10000][0],newLayout[10000][1],marker="o", markersize=20, markeredgecolor="red", markerfacecolor="green")

    for point in newLayout:
        plt.plot(point[0],point[1],marker="o", markersize=10, markeredgecolor="red", markerfacecolor="green")

    plt.show()
    print("Number of Turbines:", len(newLayout))

In [ ]:
def testSolution(x,y,sh):
    return createSol(x,y,sh)

In [ ]:
myLayout=0
def testLayout(sol):
    solution=sol
    eval.initialise(sc)
    allZeros=True
    for i in range(0,len(grid)):
        if(solution[i]):
            allZeros=True
            break
    #isBestImp=False
    nturbines=0
    for i in range(len(grid)):
        if(solution[i]):
            nturbines+=1
    layout=np.empty((nturbines,2))
    l_i=0
    for i in range(len(grid)):
        if(solution[i]):
            layout[l_i][0]=grid[i][0]
            layout[l_i][1]=grid[i][1]
            l_i+=1
    return layout


### Defining the Low Level Heuristics

In [ ]:
def incX(x):
    x+=1
    return x
def decX(x):
    x-=1
    return x
def incY(y):
    y+=1
    return y
def decY(y):
    y-=1
    return y
def incShift(sh):
    sh+=5
    return sh
def decShift(sh):
    sh-=5
    return sh

# def incX(sol):
#     x=0
#     newSolution=[False]*len(grid)
#     for i in range(0,rows):
#         flag=False
#         for j in range(0,cols):
#             if sol[int(solMat[i][j])] and (solMat[i][j])!=-1:
#                 if (i+x) <rows and solMat[i+x][j] != -1:
#                     newSolution[int(solMat[i+x][j])]=True
#                     flag=True
#         if flag:
#             x=x+1
#     return newSolution


# def incY(sol):
#     x=0
#     newSolution=[False]*len(grid)
#     for i in range(0,cols):
#         flag=False
#         for j in range(0,rows):
#             if sol[int(solMat[j][i])] and (solMat[j][i])!=-1:
#                 if (i+x) <cols and solMat[j][i+x] != -1:
#                     newSolution[int(solMat[j][i+x])]=True
#                     flag=True
#         if flag:
#             x=x+1
#     return newSolution

# def decX(sol):
#     x=0
#     newSolution=[False]*len(grid)
#     for i in range(0,rows):
#         flag=False
#         for j in range(0,cols):
#             if sol[int(solMat[i][j])] and (solMat[i][j])!=-1:
#                 if (i+x) >=0 and solMat[i+x][j] != -1:
#                     newSolution[int(solMat[i+x][j])]=True
#                     flag=True
#         if flag:
#             x=x-1
#     return newSolution

# def decY(sol):
#     x=0
#     newSolution=[False]*len(grid)
#     for i in range(0,cols):
#         flag=False
#         for j in range(0,rows):
#             if sol[int(solMat[j][i])] and (solMat[j][i])!=-1:
#                 if (i+x) >=0 and solMat[j][i+x] != -1:
#                     newSolution[int(solMat[j][i+x])]=True
#                     flag=True
#         if flag:
#             x=x-1
#     return newSolution

# def incshiftRagged(sol):
#     x=0
#     newSolution=[False]*len(grid)
#     for i in range(0,rows):
#         flag=False
#         for j in range(0,cols):
#             needfix=False
#             if sol[int(solMat[i][j])] and (solMat[i][j])!=-1:
#                 if solMat[i][(j+x)%cols] != -1:
#                     newSolution[int(solMat[i][(j+x)%cols])]=True
#                     flag=True
#                     if(j+x>=cols):
#                         needfix=True
#                 if(needfix):
#                     newSolution=fixPos(int(solMat[i][(j+x)%cols]),newSolution)
#         if flag:
#             x=x+5
            
#     return newSolution

# def decshiftRagged(sol):
#     x=0
#     newSolution=[False]*len(grid)
#     for i in range(0,rows):
#         flag=False
#         for j in range(0,cols):
#             needfix=False
#             if sol[int(solMat[i][j])] and (solMat[i][j])!=-1:
#                 if solMat[i][(j+x)%cols] != -1:
#                     newSolution[int(solMat[i][(j+x)%cols])]=True
#                     flag=True
#                     if(j+x>=cols):
#                         needfix=True
#                 if(needfix):
#                     newSolution=fixPos(int(solMat[i][(j+x)%cols]),newSolution)
#         if flag:
#             x=x-5
            
#     return newSolution


import random
def removeTurbine(sol):
    temp=[]
    for i in range(len(grid)):
        if(sol[i]):
            temp.append(i)
    x = random.randrange(len(temp))
    sol[temp[x]]=False
    return sol

def addTurbine(sol):
    x = random.randrange(len(grid))
    sol[x]=True
    return sol
  
def removeStrip(sol):
    temp=[]
    for i in range(len(grid)):
        if(sol[i]):
            temp.append(i)
    x = random.randrange(len(temp))
    if temp[x]+1000 < len(sol):
      for i in range(temp[x],temp[x]+1000):
          sol[i]=False
    else:
      for i in range(temp[x],temp[x]-1000,-1):
          sol[i]=False
    return sol

def removeStripAdd(sol):
    temp=solMat.T
    x=np.random.choice(range(cols),2,replace=False)
    y=np.random.choice(range(rows),2,replace=False)
    x.sort()
    y.sort()
    patch=temp[x[0]:x[0]+300,y[0]:y[0]+300]
    available=[]
    for i in range(patch.shape[0]):
        for j in range(patch.shape[1]):
            if (patch[i][j] !=-1):
                sol[int(patch[i][j])]=False
                available.append(int(patch[i][j]))
#     newTurb=np.random.choice(available,5,replace=False)
#     for i in newTurb:
#         solution[i]=True
    cons=True
    a=available.copy()
    while(cons):
        ch=random.choice(a)
        sol[ch]=True
        nturbines=0
        for i in available:
            if(sol[i]):
                nturbines+=1
        lay=np.empty((nturbines,2))
        l_i=0
        for i in available:
            if(sol[i]):
                lay[l_i][0]=grid[i][0]
                lay[l_i][1]=grid[i][1]
                l_i+=1 
        cons=eval.checkConstraints(lay)
        if(not cons):
            sol[ch]=False
        a.remove(ch)
    return sol


# Optimization

### Defining ACO-HH operations

In [ ]:
def savelocalImp(h,oldObj,newObj):
    if llhProp[h][4]>0:
        llhProp[h][0]=llhProp[h][0]+(oldObj-newObj)

In [ ]:
def saveglobalImp(h,oldObj,newObj):
    for i in h:
        llhProp[i][1]=llhProp[i][1]+(abs(oldObj-newObj))

In [ ]:
def incLLH(h):
    llhProp[h][4]+=1

In [ ]:
def updateLLHProp(llhProp):
    minMkl=min(llhProp[0:,0])
    minMCkh=min(llhProp[0:,1])
    for i in range(len(llhProp)):
        eeta=llhProp[i][0]+1-minMkl
        llhProp[i][2]=round(eeta,2)
        tau=llhProp[i][1]+1-minMCkh
        llhProp[i][3]=round(tau,2)
    den=0
    for i in range(len(llhProp)):
        den+=(pow(llhProp[i][1],alpha)*pow(llhProp[i][0],beta))
    for h in range(len(llhProp)):
        p=(pow(llhProp[h][1],alpha)*pow(llhProp[h][0],beta))/den
        llhProp[h][5]=p
    return llhProp

In [ ]:
def chooseLLH():
    return np.random.choice([0,1,2,3,4,5,6,7], 1,p = llhProp[0:,5])[0]
def randChoice():
    return np.random.choice([0,1,2,3,4,5,6], 1)[0]

### Performing Optimisation

In [ ]:
avgObjs=[]
avgLLH=[]
bestRunOpt=[]
allRunObjs=[]
bestRunObj=999
bestRunSol=[]

In [ ]:
for g in range(3):
    print("\n********** RUN ",g," **********\n")    
    nAnts=5
    conSteps=5
    maxIter=4
    alpha=1
    beta=2
    bestObj=9999
    objs=[]
    bestSol=[]
    bestLLH=[]

    stepX=0
    stepY=0
    shift=0
    st = time.time()
    solution=initialSol(stepX,stepY,shift)
    obj=evaluate(solution)
    objs.append(obj)
    antProp=np.empty((nAnts,4),dtype=object)
    for i in range(nAnts):
        antProp[i]=np.array([solution,[],obj,9999],dtype=object)
    llhProp=np.empty((8,6),dtype=np.float32)
    for i in range(8):
        llhProp[i]=np.array([0,0,0,0,0,1/8])



    queue=[]
    for i in range(maxIter):
        print("\nIteration ",i," :")
        print("Best sol at start of Iteration: ",bestObj,"\n")
        LLHit=[]
        for k in range(nAnts):
            antProp[k][1]=[]

        for j in range(conSteps):
            print("\nConstruction Step ",j," :")
            
            for k in range(nAnts):
                print("\nAnt ",k," :\n")
                
                oldSol=antProp[k][0]
                oldObj=antProp[k][2]
                if j==0:
                    antProp[k][3]=oldObj
                print("\nAnt Prop at Start:\n",antProp[:,1:])
                print("Best Solution of ant : ",antProp[k][3],"\n")
                #print("Ant Prop at start:\n",antProp[:,1:],"\n")
                
                h=chooseLLH()
                incLLH(h)
                #h=randChoice()
                print("Chosen Heuristic: ",h)
                antProp[k][1].append(h)
                newSol=createSol(antProp[k][1],bestLLH)
                print("Applied Heuristics: ",antProp[k][1]+bestLLH)
                queue.insert(0,h)
                if len(queue)>=10:
                    print("Sat Cond: ",len(set(queue)))
                    if len(set(queue))<=2:
                        print("Reached Saturation")
                        llhProp[0:,5]=np.array([1/8]*8)
                        queue=[]
                    else:
                        queue.pop()
                
                if eval.checkConstraints(testLayout(newSol)):
                    newObj=evaluate(newSol)
                    print("Solution of ant after construction: ",newObj,"\n")
                    savelocalImp(h,oldObj,newObj)
                    if newObj<=oldObj+0.01*oldObj:
                        antProp[k][0]=newSol
                        antProp[k][2]=newObj
                        if antProp[k][2]<bestObj:
                            bestSol=antProp[k][0]
                            bestObj=antProp[k][2]
                            LLHit=antProp[k][1]
                    else:
                      antProp[k][1].pop()
                    
                else:
                    antProp[k][0]=oldSol
                    savelocalImp(h,oldObj,oldObj+0.000003)
                    antProp[k][1].pop()
                
                #print("AntProp at End: \n",antProp[:,1:],"\n")
                
                
                
        for k in range(nAnts):
            saveglobalImp(antProp[k][1],antProp[k][3],antProp[k][2])
        bestLLH=bestLLH+LLHit

        
        print("Best LLH: ",bestLLH,"\n")
        llhProp=updateLLHProp(llhProp)
        
        for k in range(nAnts):
            antProp[k][0]=bestSol
            antProp[k][1]=LLHit
            antProp[k][2]=bestObj

            
        
        objs.append(bestObj)
        
        print("Best sol at end of Iteration: ",bestObj,"\n")
    et = time.time()
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')

    # Storing Run results
    avgObjs.append(min(objs))
    allRunObjs.append(objs)
    avgLLH.append(bestLLH)
    if min(objs)<bestRunObj:
      bestRunObj=min(objs)
      bestRunOpt=objs
      bestRunSol=bestSol

# Visualising Optimisation Results

In [ ]:
#Get best objective values got of all runs
avgObjs

In [ ]:
#get the best objective value out of all runs
bestRunObj

In [ ]:
#Line graphs showing the change in objective values vs iterations
plt.figure(figsize=(6,4))
plt.title("Ants: 5  Construction Steps: 5",fontsize=20)
plt.xlabel("Iterations",fontsize=15)
plt.ylabel("Cost of Energy",fontsize=15)
plt.yscale("logit")
for i in range(len(allRunObjs)):
  plt.plot(allRunObjs[i],label="Run "+str(i+1))
plt.legend()

In [ ]:
#Create Pie chart of LLH usage
allLLH=[0]*8
for i in avgLLH:
  for j in i:
    allLLH[j]+=1
allLLH
plt.figure(figsize=(10,10))
plt.title("Secnario 3",fontsize=15)
plt.pie(allLLH,autopct='%1.1f%%')
plt.legend(labels=["LLH1","LLH2","LLH3","LLH4","LLH5","LLH6","LLH7","LLH8"])
plt.show()

In [ ]:
#Visualise the best layout produced out of all runs
visualize(bestRunSol)